# Condition Variables

We often need to have a thread wait for some condition to be true before it can continue executing.

One thread might need to wait for another thread to complete some action before it can continue with its execution.

A similar (but not exact) example is a parent waiting for a child.

We accomplish this with **condition variables**.

A **condition variable** is a queue of waiting threads.

We always need a lock associated with a condition variable, and we have two concurrency functions we use to interact with condition variables:

- `wait(cond_t *cv, lock_t *mutex)`
    - it is assumed that the thread calling wait holds the lock when it is called.
    - adds the thread to the CV queue, puts the caller to sleep, releasing the lock.
    - when woken, `wait()` then reaquires the lock and returns, allowing this thread to continue
- `signal(cond_t *cv)`
    - signals that one thread waiting on the condition variable can wake up and execute



# Example: Implementing Join

We'll implement the ability to have a parent wait for a child.

```C
int done  = 0;
pthread_mutex_t m = PTHREAD_MUTEX_INITIALIZER;
pthread_cond_t c  = PTHREAD_COND_INITIALIZER;

void thr_exit() {
    Pthread_mutex_lock(&m);
    done = 1;
    Pthread_cond_signal(&c);
    Pthread_mutex_unlock(&m);
}

void *child(void *arg) {
    printf("child\n");
    thr_exit();
    return NULL;
}

void thr_join() {
    Pthread_mutex_lock(&m);
    while (done == 0)
        Pthread_cond_wait(&c, &m);
    Pthread_mutex_unlock(&m);
}

int main(int argc, char *argv[]) {
    printf("parent: begin\n");
    pthread_t p;
    Pthread_create(&p, NULL, child, NULL);
    thr_join();
    printf("parent: end\n");
    return 0;
}
```

Do we need the `done` variable?

```C
void thr_exit() {
    Pthread_mutex_lock(&m);
    Pthread_cond_signal(&c);
    Pthread_mutex_unlock(&m);
}

void thr_join() {
    Pthread_mutex_lock(&m);
    Pthread_cond_wait(&c, &m);
    Pthread_mutex_unlock(&m);
}
```

## Condition Variables Best Practices

1) Always grab the lock before calling on wait and release it ASAP afterwards.
2) Always recheck the condition after calling on wait.
    - the state of the condition might have changed between being signaled and reacquiring the lock within `wait()`
3) Always grab the lock before `signal()` and release it ASAP afterwards.

# The Producer/Consumer Model

The producer adds information to some shared buffer.

The consumer retrieves information from the shared buffer.

## Practical Example: A Web Server!!

The producer accepts and adds web requests to a shared queue.

Consumers remove, process, and respond to the web requests from the shared queue.

### Condition Variables

Condition variables are used to ensure that there are web requests on the queue for the consumers to consume (otherwise consumers `wait()` for requests to be added to the queue).

A condition variable is also used to ensure that the queue is not full.

If the queue is empty, consumers will `wait()` until signaled by producers to continue.

If the queue is full, the producers must `wait()` until signaled by consumers to continue.

## A second example: Shell Pipes

The shell pipe uses the producer/consumer model.

`cat source.c | wc -l`

Pipe the output of `cat source.c` into the input of `wc -l`

`wc` must wait until there is input in its buffer before it can start executing.



# Implemeting Producer/Consumer

```c
int buffer;
int count = 0; //initially, empty

void put(int value) {
  assert(count == 0); // can only put a value into an empty buffer!
  count = 1; // now buffer will be full
  buffer = value;
}

void get() {
  assert(count == 1); // can only get a value from non-empty buffer!
  count = 0; // now buffer will be empty
  return buffer;
}
```

Our buffer holds a single value. Could expand it to be an array of values.

If the assert statements fail, the threads will quit. Our job is to ensure they can never fail, eg, the producer will never try to put a value in a full buffer, and a consumer will never try to get a value from an empty buffer.

The producer and consumer:

```C
// This will not work! The threads are not synchronized
int loops;

void *producer(void *args) {
  int i;
  for (i = 0; i < loops; i++) {
    put(i);
  }
}

void *consumer(void *args) {
  while(1){
    int tmp = get();
    printf("%d\n", tmp);
  }
}
```

## Synchronizing Threads

```C
// This will not work! Both producer and consumers wait on same
// condition variable. A consumer can signal another consumer to
// get a value from the empty buffer
int loops;
cond_t cond;
lock_t mutex;

void *producer(void *args) {
  int i;
  for (i = 0; i < loops; i++) {
    pthread_mutex_lock(&mutex);         // p1
    if(count == 1){                     // p2
      pthread_cond_wait(&cond, &mutex); // p3
    }
    put(i);                             // p4
    pthread_cond_signal(&cond);         // p5
    pthread_mutex_unlock(&mutex);       // p6
  }
}

void *consumer(void *args) {
  while(1){
    pthread_mutex_lock(&mutex);         // c1
    if(count == 0){                     // c2
      pthread_cond_wait(&cond, &mutex); // c3
    }
    int tmp = get();                    // c4
    pthread_cond_signal(&cond);         // c5
    pthread_mutex_unlock(&mutex);       // c6
    printf("%d\n", tmp);
  }
}
```

We have two conditions (buffer full vs buffer empty), but one condition variable. This is a problem.

Suppose we have 1 producer, but two consumers.

```                                              | C1 signals C2 to wake up
C1: c1 c2 c3                                 c4 c5 c6
C2:          c1 c2 c3                                  c4 (fails! buffer empty)
P :                   p1 p2 p4 p5 p6 p1 p2 p3
```

```C
// This will not work! The threads are not synchronized
int loops;
cond_t full;
cond_t empty;
lock_t mutex;

void *producer(void *args) {
  int i;
  for (i = 0; i < loops; i++) {
    pthread_mutex_lock(&mutex);          // p1
    if(count == 1){                      // p2
      pthread_cond_wait(&empty, &mutex); // p3
    }
    put(i);                              // p4
    pthread_cond_signal(&full);          // p5
    pthread_mutex_unlock(&mutex);        // p6
  }
}

void *consumer(void *args) {
  while(1){
    pthread_mutex_lock(&mutex);         // c1
    if(count == 0){                     // c2
      pthread_cond_wait(&full, &mutex); // c3
    }
    int tmp = get();                    // c4
    pthread_cond_signal(&empty);        // c5
    pthread_mutex_unlock(&mutex);       // c6
    printf("%d\n", tmp);
  }
}
```